In [ ]:
import cv2
import os
import sys
import random
from matplotlib import pyplot as plt
import numpy as np
sys.path.append('../modeling')
from baseline_model import baseline_model
from tensorflow.keras.utils import to_categorical

In [ ]:
corr_masks = "../data/original_data/masks/"
uncorrected_masks = "../data/uncorrected_data/"

img_list = os.listdir(corr_masks)
img_list.sort()
msk_list = os.listdir(uncorrected_masks)
msk_list.sort()
num_images = len(os.listdir(corr_masks))
print("Total number of training images are: ", num_images)

In [ ]:
img_num = random.randint(0, num_images-1)

corr_image_for_plot = cv2.imread(corr_masks+img_list[img_num], 1)
uncorr_image_for_plot =cv2.imread(uncorrected_masks+msk_list[img_num], 1)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(uncorr_image_for_plot, cmap='gray')
plt.title('Uncorrected Image')
plt.subplot(122)
plt.imshow(corr_image_for_plot, cmap='gray')
plt.title('Corrected Image')
plt.show()

In [ ]:
baseline_model(corr_masks, uncorrected_masks)

In [ ]:
def baseline_model(corr_masks, uncorrected_masks):
    list = []
    for correct in glob.glob(corr_masks + "*.tif"):
        for uncorrect in glob.glob(uncorrected_masks + "*.tif"):
            img_1 = cv2.imread(correct, 1)
            img_2 = cv2.imread(uncorrect, 1)

            res = cv2.absdiff(img_1, img_2)
            res = res.astype(np.uint8)
            percentage = np.round(100 - ((np.count_nonzero(res) * 100) / res.size), 2)
            list.append(percentage)
            print(f"The mean accuracy is equal to {np.mean(list)}")
            print(f"These are the accuracies of the individual images{list}")

### Baseline model for IoU

In [ ]:
# We need the images as w,h,classes
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
h, w = corr_image_for_plot.shape  

mask_for_plot_reshaped = corr_image_for_plot.reshape(-1,1)
mask_for_plot_reshaped_encoded = labelencoder.fit_transform(mask_for_plot_reshaped)
corr_images_encoded = mask_for_plot_reshaped_encoded.reshape(h, w)

mask_for_plot_reshaped = uncorr_image_for_plot.reshape(-1,1)
mask_for_plot_reshaped_encoded = labelencoder.fit_transform(mask_for_plot_reshaped)
uncorr_images_encoded = mask_for_plot_reshaped_encoded.reshape(h, w)


In [ ]:
corr_image_for_plot.shape

In [ ]:
uncorr_images_encoded.shape

In [ ]:
def preprocess_data(mask, num_class):
    #Convert mask to one-hot
    labelencoder = LabelEncoder()
    h, w, c = mask.shape  
    mask = mask.reshape(-1,1)
    mask = labelencoder.fit_transform(mask)
    mask = mask.reshape(h, w, c)
    mask = to_categorical(mask, num_class)
      
    return (mask)

In [ ]:
uncorr_images_encoded = preprocess_data(uncorr_image_for_plot, 4)

In [ ]:
corr_images_encoded = preprocess_data(uncorr_image_for_plot, 3)

In [ ]:
uncorr_images_encoded.shape

In [ ]:
# one hot encoding of predictions
#num_classes = y_pred.shape[-1]
#y_pred = np.array([ np.argmax(y_pred, axis=-1)==i for i in range(num_classes) ]).transpose(1,2,3,0)

axes = (0,1) # W,H axes of each image
intersection = np.sum(np.logical_and(uncorr_images_encoded, corr_images_encoded), axis=axes)
# intersection = np.sum(np.abs(y_pred * y_true), axis=axes)
union = np.sum(np.logical_or(uncorr_images_encoded, corr_images_encoded), axis=axes)
mask_sum = np.sum(np.abs(corr_images_encoded), axis=axes) + np.sum(np.abs(uncorr_images_encoded), axis=axes)
# union = mask_sum  - intersection

smooth = .001
iou = (intersection + smooth) / (union + smooth)
dice = 2 * (intersection + smooth)/(mask_sum + smooth)

iou = np.mean(iou)
dice = np.mean(dice)

In [ ]:
dice